In [1]:
using Pkg
using PyCall
using TickTock
using Dates

In [2]:
np=pyimport("numpy")

PyObject <module 'numpy' from 'C:\\Users\\bhaiyasingh\\.julia\\conda\\3\\lib\\site-packages\\numpy\\__init__.py'>

In [3]:
## rolette_wheel function to choose the best node based on probability

In [18]:
function rolette_wheel(p)
    
    cumsum = np.cumsum(p)

    r = rand()

    next_node = findall(x->x >= r,cumsum)

    next_node = next_node[1]

    return next_node
    
end

rolette_wheel (generic function with 1 method)

In [19]:
## create colony function

In [20]:
function create_colony(graph, ant_no, tau, eta, alpha, beta)
    
    n = size(graph)[1]
    
#     println(n)

    colony = []
    

    for i in 1:ant_no
        
        tour = []
        
        initial_node = rand(1:n)  # select a random node

        append!(tour,initial_node)

        
        P_allNodes = (np.power(tau,alpha)) * (np.power(eta,beta))
        
#         println(P_allNodes)
        
        p = P_allNodes[initial_node,:]

        for j in 2:n  # choose the rest of nodes

            
            p[tour[end]] = 0  # assign 0 to already visited node
            

            P = p / np.sum(p)

            next_node = rolette_wheel(P)
            
            append!(tour,next_node)
            
        end

        append!(tour,tour[1])
        
        append!(colony,[tour])
        
    end
            

    return colony
            
end

create_colony (generic function with 1 method)

In [21]:
function fitness(tour, graph)
    
    fitness = 0
    n=length(tour)

    for i in 1:n-1
        
        current_node = tour[i]
        next_node = tour[i+1]

        fitness = fitness + graph[current_node,next_node]
        
    end

    
    return fitness
        
end

fitness (generic function with 1 method)

In [22]:
function update_phromone(tau, colony, antno,graph)

    n=size(colony[1])[1]
    
    
    for i in 1:antno
        for j in 1:n-1
            current_node = colony[i][j]
            next_node = colony[i][j+1]
            
#             println("-----------")

            tau[current_node,next_node] = tau[current_node,next_node] + 1 / fitness(colony[i], graph)
            tau[next_node,current_node] = tau[next_node,current_node] + 1 / fitness(colony[i], graph)
            
        end
    end
    
    return tau
    
end

update_phromone (generic function with 1 method)

In [46]:

function aco_algo(graph)
    

    max_iter = 1000
    ant_no = 10
    n = size(graph)[1]

    tau0 = 10 * 1 / (n * np.mean(l))  # initial phromone

    tau = tau0 * np.ones((n, n))  # phromone matx

    eta = 1./ graph  # desirability of each edge


    rho = 0.05  # evporation rate
    alpha = 1  # phromone exponential parameters
    beta = 1  # desirability exponential parameter

    best_fitness = np.inf
    
    best_tour=[]

    # main loop for ACO
    for i in 1:max_iter

        all_fitness = []

        colony = create_colony(l, ant_no, tau, eta, alpha, beta)

        for j in 1:ant_no
            
            append!(all_fitness,fitness(colony[j], graph))

        end

        min_tour =  colony[findmin(all_fitness)[end]]
        
        min_fitness = all_fitness[findmin(all_fitness)[end]]
        


        if min_fitness < best_fitness
            best_fitness = min_fitness
            best_tour = colony[findmin(all_fitness)[end]]
        end
        
         

        tau = update_phromone(tau, colony, ant_no, graph)

        tau = (1 - rho) * tau  # applying evaporation

        println("Iteration = $(i), shortest_path =$(best_fitness) , best_tour=$(best_tour)")
        
    end
    
end
        


LoadError: syntax: invalid syntax "1./"; add space(s) to clarify

In [47]:
l = [[  0, 290, 250,  230,  190,  334, 365,   40], # Dhaka 1
    [290,   0, 337,  453,  396,  560, 581,  244], # Syhlet 2
    [250, 337,   0,  495,  396,  540, 120,  240], # Chittagonj 3
    [230, 453, 495,    0,  360,  150, 595,  242], # Rajshahi 4
    [190, 396, 396,  360,    0,  356, 496,  253], # Jossore 5
    [334, 560, 540,  150,  356,    0, 674,  275], # Dinajpur 6
    [365, 581, 120,  595,  496,  674,   0,  397], # Coxsbazar 7
    [40,  244, 240,  242,  253,  275, 397,    0]] # Narsingdi 8
    
l=np.array(l)


8×8 Matrix{Int64}:
   0  290  250  230  190  334  365   40
 290    0  337  453  396  560  581  244
 250  337    0  495  396  540  120  240
 230  453  495    0  360  150  595  242
 190  396  396  360    0  356  496  253
 334  560  540  150  356    0  674  275
 365  581  120  595  496  674    0  397
  40  244  240  242  253  275  397    0

In [48]:
tick()
a=time()
aco_algo(l)
b=time()
println(b-a)
tok()

Iteration = 1, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 2, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 3, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 4, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 5, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 6, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 7, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 8, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 9, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 10, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 11, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 12, shortest_path =2031 , best_tour=Any[1, 2, 3, 7, 5, 6, 4, 8, 1]
Iteration = 13, shortest_path =2031 , best_tour=Any[1, 2, 3, 

Iteration = 130, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 131, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 132, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 133, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 134, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 135, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 136, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 137, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 138, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 139, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 140, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 141, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 142, shortest_path =1973 , b

Iteration = 259, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 260, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 261, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 262, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 263, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 264, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 265, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 266, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 267, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 268, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 269, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 270, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 271, shortest_path =1973 , b

Iteration = 387, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 388, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 389, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 390, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 391, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 392, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 393, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 394, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 395, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 396, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 397, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 398, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 399, shortest_path =1973 , b

Iteration = 515, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 516, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 517, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 518, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 519, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 520, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 521, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 522, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 523, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 524, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 525, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 526, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 527, shortest_path =1973 , b

Iteration = 643, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 644, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 645, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 646, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 647, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 648, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 649, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 650, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 651, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 652, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 653, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 654, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 655, shortest_path =1973 , b

Iteration = 771, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 772, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 773, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 774, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 775, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 776, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 777, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 778, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 779, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 780, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 781, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 782, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 783, shortest_path =1973 , b

Iteration = 899, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 900, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 901, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 902, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 903, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 904, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 905, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 906, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 907, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 908, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 909, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 910, shortest_path =1973 , best_tour=Any[4, 6, 5, 7, 3, 2, 8, 1, 4]
Iteration = 911, shortest_path =1973 , b

┌ Info:  started timer at: 2021-12-15T17:15:09.249
└ @ TickTock C:\Users\bhaiyasingh\.julia\packages\TickTock\RsTHR\src\TickTock.jl:32


2.734027

In [43]:
l

8×8 Matrix{Int64}:
   0  290  250  230  190  334  365   40
 290    0  337  453  396  560  581  244
 250  337    0  495  396  540  120  240
 230  453  495    0  360  150  595  242
 190  396  396  360    0  356  496  253
 334  560  540  150  356    0  674  275
 365  581  120  595  496  674    0  397
  40  244  240  242  253  275  397    0

In [45]:
l= 1 ./ l

8×8 Matrix{Float64}:
 Inf           0.00344828   0.004       …   0.00273973   0.025
  0.00344828  Inf           0.00296736      0.00172117   0.00409836
  0.004        0.00296736  Inf              0.00833333   0.00416667
  0.00434783   0.00220751   0.0020202       0.00168067   0.00413223
  0.00526316   0.00252525   0.00252525      0.00201613   0.00395257
  0.00299401   0.00178571   0.00185185  …   0.00148368   0.00363636
  0.00273973   0.00172117   0.00833333     Inf           0.00251889
  0.025        0.00409836   0.00416667      0.00251889  Inf